#Face Mask Detection Model

In [1]:
%pip install kagglehub

In [57]:
# นำเข้า modules ของ libraries
import os
import re
import numpy as np
from PIL import Image
import xml.etree.ElementTree as ET
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from kagglehub import login, dataset_download
from pandas import DataFrame

In [3]:
# Login เข้า Kaggle (ต้องดาวโหลด์ใบ credentail จาก kaggle ก่อนแล้วเอา username กับ key มากรอก)
# หรือถ้าไม่มีให้ใช้ข้อมูลด้านล่างต่อไปนี้

# ให้นำค่า value ไปกรอกในฟอร์มของ kaggle
kaggle_api = {
    "username":"warinsaipanya",
    "key":"e9d6ec16f057b50fac90908de9990b27"
}

login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [4]:
# ดาวโหลด์ตัว datasets จาก kaggle เก็บไว้ใน path ที่ระบุ
path = dataset_download("andrewmvd/face-mask-detection")
print(f"path ของไฟล์ dataset อยู่ที่: {path}")

path ของไฟล์ dataset อยู่ที่: /root/.cache/kagglehub/datasets/andrewmvd/face-mask-detection/versions/1


In [5]:
# classes เป็น list ไว้เก็บคำตอบ
classes = ["mask worn incorrectly", "with mask", "without mask"]

In [38]:
# ชื่อ models ทั้งหมด
model_names = ("Neural Network", "KNN", "Desicion Tree", "Random Forest")

# สร้างตัวแปรเก็บเป็น list ไว้เก็บค่าตัวเลขผลลัพธ์ของ model เพื่อนำไป plot graph และ ทำตาราง
accuracy_values = []
precision_values = []
recall_values = []

# function สำหรับเพิ่มค่า element ให้ตัวแปร list ด้านบน
def add_elements(accuracy, precision, recall):
  accuracy_values.append(accuracy)
  precision_values.append(precision)
  recall_values.append(recall)

In [6]:
# path ที่เก็บข้อมูลรูปภาพไว้ train
folder_path = f"{path}/images"
# path ที่เก็บข้อมูลไฟล์ xml
xml_folder = f"{path}/annotations"

In [12]:
# สร้าง list ไว้เก็บข้อมูล x และ y
# x สำหรับเก็บข้อมูล features ของรูปภาพ
x = []
# y สำหรับเก็บคำตอบของรูปภาพ
y = []

In [13]:
# function สำหรับการอ่านข้อมูลในไฟล์ xml
def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    # เก็บข้อมูลใน tag ของ object
    objects = []

    # วน loop หา element ที่มีชื่อว่า object และได้กลับคืนมาเป็น list แล้ววน loop
    for obj in root.findall("object"):
        # หา tag ที่ชื่อ name แล้วอ่าน content ด้านในเก็บ class คำตอบของรูป
        label = obj.find("name").text
        # หา tag bndbox
        bndbox = obj.find("bndbox")
        # อ่านข้อมูลด้านใน element bndbox แล้วแปลงค่าเป็น int
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)
        # เพื่อ dictionary เข้าไปใน list
        objects.append({"label": label, "bbox": [xmin, ymin, xmax, ymax]})

    return objects

In [14]:
def extract_number(filename):
    match = re.search(r"\d+", filename)
    return int(match.group()) if match else -1

In [15]:
# list เก็บชื่อไฟล์ xml แบบเรียงลำดับชื่อไฟล์เป็นไฟล์ข้อมูลสำหรับแต่หล่ะรูปภาพ
xml_files = sorted(os.listdir(xml_folder), key=extract_number)
# สร้าง object ของ label encoder ไว้ normalize ตัว label ของรูป
encoder = LabelEncoder()

# แสดงผล list ที่เก็บไฟล์ xml ไว้
print(xml_files)

['maksssksksss0.xml', 'maksssksksss1.xml', 'maksssksksss2.xml', 'maksssksksss3.xml', 'maksssksksss4.xml', 'maksssksksss5.xml', 'maksssksksss6.xml', 'maksssksksss7.xml', 'maksssksksss8.xml', 'maksssksksss9.xml', 'maksssksksss10.xml', 'maksssksksss11.xml', 'maksssksksss12.xml', 'maksssksksss13.xml', 'maksssksksss14.xml', 'maksssksksss15.xml', 'maksssksksss16.xml', 'maksssksksss17.xml', 'maksssksksss18.xml', 'maksssksksss19.xml', 'maksssksksss20.xml', 'maksssksksss21.xml', 'maksssksksss22.xml', 'maksssksksss23.xml', 'maksssksksss24.xml', 'maksssksksss25.xml', 'maksssksksss26.xml', 'maksssksksss27.xml', 'maksssksksss28.xml', 'maksssksksss29.xml', 'maksssksksss30.xml', 'maksssksksss31.xml', 'maksssksksss32.xml', 'maksssksksss33.xml', 'maksssksksss34.xml', 'maksssksksss35.xml', 'maksssksksss36.xml', 'maksssksksss37.xml', 'maksssksksss38.xml', 'maksssksksss39.xml', 'maksssksksss40.xml', 'maksssksksss41.xml', 'maksssksksss42.xml', 'maksssksksss43.xml', 'maksssksksss44.xml', 'maksssksksss45.xml

In [16]:
# วน loop แต่ล่ะชื่อ xml file
for file in xml_files:
    # เอา path ของที่เก็บไฟล์ xml  มา join กับชื่อไฟล์ จะได้ตำแหน่ง path ที่แท้จริงของไฟล์ xml
    file_path = os.path.join(xml_folder, file)

    # เช็คตรวจสอบไฟล์ว่ามี path ไฟล์นั้นอยู่จริงไหม หรือไฟล์นั้นไม่ใช่ไฟล์ นามสกุล .xml ให้ข้าม loop นั้น
    if not os.path.isfile(file_path) or not file.lower().endswith('.xml'):
        continue

    try:
        # แปลงไฟล์ นามสกุลลงท้าย .xml เป็น .png ชื่อไฟล์ xml กับ ชื่อรูปภาพเหมือนกันต่างกันแค่คนหล่ะนามสกุลไฟล์
        image_name = file.replace(".xml", ".png")
        # เอา path ที่เก็บรูปภาพมา join กับ ชื่อไฟล์รูปภาพที่พึ่งแก้นามสกุลไฟลืไป ได้เป็น path ของรูปภาพที่แท้จริง
        image_path = os.path.join(folder_path, image_name)

        # เช็คเงื่อนไขถ้า path ของรูปภาพไม่มีอยู่ ให้ข้าม loop นั้นไป
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            continue

        # เปิดรูปภาพตามตำแหน่งของ path รูปภาพแล้วแปลงรูปภาพนั้นให้เป็น grayscale
        img = Image.open(image_path).convert('L')
        # ปรับขนาดของรูปภาพที่ size หลายขนาดให้เป็นขนาด 512 x 512 (height = width)
        img = img.resize((512, 512))
        # เพิ่ม element เข้าไปใน list โดยให้รูปภาพปรับขนาดรูปร่างความกว้างและความสูงเป็น 512 และ มีช่องสีแค่ 1
        x.append(np.array(img).reshape(512, 512, 1))

        # เรียกใช้ function ได้เป็น list ที่เก็บ dictionary ที่เป็นคุณสมบัติของรูปภาพ
        objects = parse_xml(file_path)

        # เช็คถ้า objects นั้นไม่ใช้ list ว่าง (มี elements อยู่ใน list) ให้ทำเงื่อนไขด้านล่างนี้
        if objects:
            # อ่านคำตอบมาแค่ 1 คนจากรูปภาพนั้น
            # ให้ตัวแปร y เก็บ class ของคำตอบแต่ล่ะรูป
            y.append(objects[0]["label"])
        else:
            print(f"Warning: No objects found in {file_path}")
            continue
    except Exception as e:
        print(f"Error processing {file}: {e}")

# เช็คถ้าข้อมูลนั้นไม่มีอยู่ให้โยน exception นี้ออกไป
if len(x) == 0 or len(y) == 0:
    raise ValueError("No data found! Check your dataset paths and XML annotations.")

# encoded ตัวคำตอบ labels แปลงจาก string -> int ค่าอยู่ในช่วง 0 - 2
y = encoder.fit_transform(y)
# แปลงข้อมูล array จากชนิด unsigned int -> float และหารด้วย 255 เพิ่อแปลงเป็น grayscale
x = np.array(x).astype('float32') / 255
# แบ่งข้อมูล train และ test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [18]:
# แสดงผลขนาดรูปร่างของข้อมูล train และ test
# จำนวนข้อมูลรูปภาพมีทั้งหมด 853 รูป แบ่งเป็น train 682 รูป และ test 171 รูป
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(682, 512, 512, 1)
(171, 512, 512, 1)
(682,)
(171,)


### Neural Netwoerk Model

In [19]:
# สร้าง object ของ model
model = Sequential()

# เพิ่มแต่ล่ะ layers ให้ model
model.add(Flatten(input_shape=(512, 512, 1)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# train model 14 รอบ
model.fit(x_train, y_train, epochs=14)

Epoch 1/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 0.6319 - loss: 143.1385
Epoch 2/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 0.7464 - loss: 29.3112
Epoch 3/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.6596 - loss: 4.5866
Epoch 4/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 0.8091 - loss: 1.0655
Epoch 5/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 98s 4s/step - accuracy: 0.8156 - loss: 0.9798
Epoch 6/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 70s 3s/step - accuracy: 0.8051 - loss: 0.9497
Epoch 7/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 71s 3s/step - accuracy: 0.8080 - loss: 0.8557
Epoch 8/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 92s 3s/step - accuracy: 0.7867 - loss: 0.8547
Epoch 9/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 0.8111 - loss: 0.7827
Epoch 10/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - accuracy: 0.7895 - loss: 0.8011
Epoch 11/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - accuracy: 0.7983 - loss: 0.7514
Epoch 12/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.8015 -

In [23]:
# บันทึกตัว model เป็นไฟล์ .keras
model.save('/content/face_mask_detection_model.keras')

In [39]:
# คำนวณค่า accuracy และ ค่า loss ของ model
loss, accuracy = model.evaluate(x_test, y_test)

# แสดงผลค่าออกมา
print(f"Accuracy: {accuracy}")
print(f"Loss: {loss}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - accuracy: 0.8609 - loss: 0.5239
Accuracy: 0.8830409646034241
Loss: 0.5003550052642822


In [40]:
# ทำนายค่าผลลัพธ์
y_pred = np.argmax(model.predict(x_test), axis=-1)

# คำนวณค่า precision และ recall
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# แสดงผลลัพธ์
print(f"Precision: {precision}")
print(f"Recall: {recall}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 359ms/step
Precision: 0.8830409356725146
Recall: 1.0


In [41]:
# เรียกใช้ function เพิ่มค่าที่สำคัญเข้าไปใน list
add_elements(accuracy, precision, recall)

In [ ]:
# โหลดตัว model
model = load_model('/content/face_mask_detection_model.keras')

In [81]:
# แปลงจาก array 4 มิติให้เป็น array 2 มิติ
x_train2 = x_train.reshape(x_train.shape[0], -1)
x_test2 = x_test.reshape(x_test.shape[0], -1)
# แปลงเป็๋น array 1 มิติ
y_train2 = y_train.ravel()
y_test2 = y_test.ravel()

In [82]:
# แสดงรูปร้่างของข้อมูล train และ test
print(x_train2.shape)
print(x_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(682, 262144)
(171, 262144)
(682,)
(171,)


### KNN Model

### Desicion Tree Model

### Random Forest Model

In [85]:
rf_model = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=42)
rf_model.fit(x_train2, y_train2)

y_pred = rf_model.predict(x_test2)
score = rf_model.score(x_test2, y_test2)

print(f"Mean Accuracy: {score}")

Mean Accuracy: 0.8771929824561403
